In [1]:
import os
import logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
from tensorflow_serving.apis import input_pb2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    print(details)

{'compute_capability': (7, 5), 'device_name': 'Tesla T4'}


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
pathtrain = "/content/drive/MyDrive/colab/TFM/datasets/MSLR-WEB10K/tfrecords/train.tfrecords"
patheval = "/content/drive/MyDrive/colab/TFM/datasets/MSLR-WEB10K/tfrecords/val.tfrecords"

Mounted at /content/drive


## Create a model builder

### Specify Features

In [4]:
names = ['covered_query_term_number_body','covered_query_term_number_anchor','covered_query_term_number_title', 
          'covered_query_term_number_url','covered_query_term_number_whole_document','covered_query_term_ratio_body',
        'covered_query_term_ratio_anchor','covered_query_term_ratio_title','covered_query_term_ratio_url',
          'covered_query_term_ratio_whole_document', 'stream_length_body', 'stream_length_anchor',
        'stream_length_title','stream_length_url','stream_length_whole_document','sum_term_freq_body','sum_term_freq_anchor','sum_term_freq_title',
          'sum_term_freq_url','sum_term_freq_whole_document','min_term_freq_body','min_term_freq_anchor','min_term_freq_title',
          'min_term_freq_url','min_term_freq_whole_document','max_term_freq_body','max_term_freq_anchor','max_term_freq_title',
          'max_term_freq_url','max_term_freq_whole_document','mean_term_freq_body','mean_term_freq_anchor','mean_term_freq_title',
          'mean_term_freq_url','mean_term_freq_whole_document','sum_stream_length_normalized_term_freq_body','sum_stream_length_normalized_term_freq_anchor',
          'sum_stream_length_normalized_term_freq_title','sum_stream_length_normalized_term_freq_url','sum_stream_length_normalized_term_whole_document',
          'min_stream_length_normalized_term_freq_body','min_stream_length_normalized_term_freq_anchor','min_stream_length_normalized_term_freq_title',
          'min_stream_length_normalized_term_freq_url','min_stream_length_normalized_term_freq_whole_document','max_stream_length_normalized_term_freq_body',
          'max_stream_length_normalized_term_freq_anchor','max_stream_length_normalized_term_freq_title','max_stream_length_normalized_term_freq_url',
          'max_stream_length_normalized_term_freq_whole_document','mean_stream_length_normalized_term_freq_body','mean_stream_length_normalized_term_freq_anchor',
          'mean_stream_length_normalized_term_freq_title','mean_stream_length_normalized_term_freq_url','mean_stream_length_normalized_term_freq_whole_document','boolean_model_body',
          'boolean_model_anchor','boolean_model_title','boolean_model_url','boolean_model_whole_document','vector_space_model_body',
          'vector_space_model_anchor','vector_space_model_title','vector_space_model_url','vector_space_model_whole_document','BM25_body',
          'BM25_anchor','BM25_title','BM25_url','BM25_whole_document','LMIR.ABS_body','LMIR.ABS_anchor','LMIR.ABS_title','LMIR.ABS_url',
          'LMIR.ABS_whole_document','LMIR.DIR_body','LMIR.DIR_anchor','LMIR.DIR_title','LMIR.DIR_url','LMIR.DIR_whole_document','LMIR.JM_body',
          'LMIR.JM_anchor','LMIR.JM_title','LMIR.JM_url','LMIR.JM_whole_document','num_slash_url','length_url','inlink_number','outlink_number',
          'pagerank','siterank','qualityscore','qualityscore2','query_url_click_count','url_click_count','url_dwell_time']


In [5]:
context_feature_spec = {}
example_feature_spec = {feat: tf.io.FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0) 
                        for feat in names}
label_spec = ('relevance_label', 
                tf.io.FixedLenFeature(shape=(1,),
                dtype=tf.int64,
                default_value=-1))

### Define input_creator

In [6]:
input_creator = tfr.keras.model.FeatureSpecInputCreator(
                context_feature_spec, example_feature_spec)

### Define preprocessor

In [7]:
def log1p(tensor):
    return tf.math.log1p(tensor * tf.sign(tensor)) * tf.sign(tensor)

preprocess_spec = {name: lambda t: log1p(t) for name in example_feature_spec.keys()}

preprocessor = tfr.keras.model.PreprocessorWithSpec(preprocess_spec)

### Define scorer

In [8]:
scorer = tfr.keras.model.GAMScorer(
        example_feature_num = 96,
        example_hidden_layer_dims = ["16","8"], 
        activation = tf.nn.relu,
        use_batch_norm = True,
        batch_norm_moment=0.99,
        dropout = 0.4
        )

### Make model_builder

In [9]:
model_builder = tfr.keras.model.ModelBuilder(
                                    input_creator=input_creator,
                                    preprocessor=preprocessor,
                                    scorer=scorer,
                                    mask_feature_name="example_list_mask",
                                    name="model_builder"
)

## Create a dataset builder

In [10]:
dataset_hparams = tfr.keras.pipeline.DatasetHparams(
                    train_input_pattern = pathtrain,
                    valid_input_pattern = patheval,
                    train_batch_size = 32,
                    valid_batch_size = 10,
                    list_size = 150,
                    dataset_reader = tfr.keras.pipeline.DatasetHparams.dataset_reader)

### Make dataset_builder

In [11]:
dataset_builder = tfr.keras.pipeline.SimpleDatasetBuilder(
                    context_feature_spec,
                    example_feature_spec,
                    mask_feature_name="example_list_mask",
                    label_spec=label_spec,
                    hparams=dataset_hparams,
                    sample_weight_spec=None)

## Create a ranking pipeline

In [12]:
pipeline_hparams = tfr.keras.pipeline.PipelineHparams(
                                          model_dir = "/content/drive/MyDrive/colab/TFM/mslr_ranking_model_dir",
                                          num_epochs = 5,
                                          steps_per_epoch = 1000,
                                          validation_steps = 100,
                                          learning_rate = 0.05,
                                          loss = 'approx_ndcg_loss',
                                          optimizer = 'adagrad')

### Define ranking_pipeline

In [13]:
ranking_pipeline = tfr.keras.pipeline.SimplePipeline(
                    model_builder,
                    dataset_builder=dataset_builder,
                    hparams=pipeline_hparams)

## Train and evaluate the model

In [14]:
ranking_pipeline.train_and_validate(verbose=1)

Epoch 1/5
1000/1000 [==============================] - 338s 338ms/step - loss: -0.6020 - metric/ndcg_1: 0.2218 - metric/ndcg_5: 0.2555 - metric/ndcg_10: 0.2869 - metric/ndcg: 0.6023 - val_loss: -0.6482 - val_metric/ndcg_1: 0.2990 - val_metric/ndcg_5: 0.3306 - val_metric/ndcg_10: 0.3605 - val_metric/ndcg: 0.6492
Epoch 2/5
1000/1000 [==============================] - 182s 182ms/step - loss: -0.6196 - metric/ndcg_1: 0.2515 - metric/ndcg_5: 0.2856 - metric/ndcg_10: 0.3172 - metric/ndcg: 0.6199 - val_loss: -0.6573 - val_metric/ndcg_1: 0.3298 - val_metric/ndcg_5: 0.3503 - val_metric/ndcg_10: 0.3782 - val_metric/ndcg: 0.6581
Epoch 3/5
1000/1000 [==============================] - 174s 174ms/step - loss: -0.6252 - metric/ndcg_1: 0.2621 - metric/ndcg_5: 0.2962 - metric/ndcg_10: 0.3278 - metric/ndcg: 0.6255 - val_loss: -0.6617 - val_metric/ndcg_1: 0.3419 - val_metric/ndcg_5: 0.3596 - val_metric/ndcg_10: 0.3871 - val_metric/ndcg: 0.6626
Epoch 4/5
1000/1000 [==============================] - 174s 1